In [1]:
import folium, heapq, copy

In [2]:
# 경북대 본관 주소
origin_lat, origin_lng = 35.89049, 128.6120
zoomLevel = 16

### 경로를 기반으로 한 경로 찾기
간선의 길이를 그림자의 비율에 따라서 조정 -> 그림자의 비율이 높을수록 길이가 짧아짐

$d_{opt} = d \times (1 - Ratio_{shadow})$

In [3]:
node_info = []
with open("nodeInfo.txt", "r") as f :
    for line in f :
        node_info.append(tuple(map(float, line.split(','))))

edges에는 가중치가 적용된 간선의 정보가 담겨있음

In [4]:
def calculate_distance(node_a, node_b) :
    return ((node_a[0] - node_b[0])**2 + (node_a[1] - node_b[1])**2)**0.5

def calculate_processed_length(node_a, node_b, ratio) :
    # 10^5는 임의의 상수(숫자가 너무 작아서 사용)
    return calculate_distance(node_a, node_b) * (1 - ratio) * 10**5 

In [5]:
edges = [[] for _ in range(len(node_info))]
non_processed_edges = [[] for _ in range(len(node_info))]
ratioMatrix = [[0 for _ in range(len(node_info))] for _ in range(len(node_info))]

with open("shadowRatio.txt", "r") as f :
    for i, line in enumerate(f) :
        for j, ratio in enumerate(line.split()) :
            ratioMatrix[i][j] = float(ratio)
            if float(ratio) < 0 : continue

            edges[i].append((j, calculate_processed_length(node_info[i], node_info[j], float(ratio))))
            non_processed_edges[i].append((j, calculate_distance(node_info[i], node_info[j])))

### 그림자 비율을 시각화한 지도

In [6]:
def color_weight(ratio) :
    if ratio < 0.1 :
        return 'red'
    elif ratio < 0.5 :
        return 'orange'
    else :
        return 'blue'
    
ratio_map = folium.Map(location=[origin_lat, origin_lng], zoom_start=zoomLevel)

for i, row in enumerate(ratioMatrix) :
    for j, ratio in enumerate(row) :
        if ratio < 0 or i >= j: continue

        folium.PolyLine(
            locations=[node_info[i], node_info[j]],
            color=color_weight(ratio),
            weight=3
        ).add_to(ratio_map)

ratio_map.save("shadowInfoMap.html")

### 다익스트라 알고리즘을 이용하여 최단 경로를 찾음

`find_shadow_path` 함수를 이용하여 최단 경로를 찾아, 그 이동경로를 반환

 - 최단 경로를 이루는 노드를 순서대로 반환

In [7]:
def find_shadow_path(stard_idx, end_idx, edge_info) :
    INF = float('inf')
    dist = [INF for _ in range(len(node_info))]
    prev = [-1 for _ in range(len(node_info))]
    dist[stard_idx] = 0
    pq = []
    heapq.heappush(pq, (0, stard_idx))

    while pq :
        cur_dist, cur_idx = heapq.heappop(pq)
        if dist[cur_idx] < cur_dist : continue
        for next_idx, next_dist in edge_info[cur_idx] :
            if dist[next_idx] > cur_dist + next_dist :
                dist[next_idx] = cur_dist + next_dist
                prev[next_idx] = cur_idx
                heapq.heappush(pq, (dist[next_idx], next_idx))

    path = []
    cur_idx = end_idx
    while cur_idx != -1 :
        path.append(cur_idx)
        cur_idx = prev[cur_idx]
    return path[::-1]

IT2호관(`144(0-idx)`)에서 글로벌플라자(`233(0-idx)`)까지의 최단 경로를 찾아보기

In [8]:
shadow_map = copy.deepcopy(ratio_map)

folium.PolyLine(
    locations = [node_info[x] for x in find_shadow_path(144, 233, edges)],
    color = 'blue',
    weight = 10,
    tooltip='그늘을 고려한 경로',
    opacity = 0.4
).add_to(shadow_map)

folium.PolyLine(
    locations = [node_info[x] for x in find_shadow_path(144, 233, non_processed_edges)],
    color = 'red',
    weight = 10,
    tooltip='단순 최단 경로',
    opacity = 0.4
).add_to(shadow_map)

shadow_map

In [9]:
shadow_map.save("shadowPathMap.html")